# Choose and Configure a Protocol

A `Protocol` describes the simulation and sampling strategy for a free energy campaign. It is specified with subclasses of the `Protocol` class, and their associated `ProtocolSettings` subclasses.

## Setup

In [1]:
from openff.units import unit

## Choose a `Protocol`

Your choice of `Protocol` determines how free energy sampling is performed. Here, we will be looking into the `RelativeHybridTopologyProtocol`:

| Name         | [RelativeHybridTopologyProtocol]         |
|:-------------|:-----------------------------------------|
| Module       | `openfe.protocols.openmm_rfe`            |
| Settings     | [RelativeHybridTopologyProtocolSettings] |  
| MD Engine    | [OpenMM]                                 |

[RelativeHybridTopologyProtocol]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.protocols.openmm_rfe.RelativeHybridTopologyProtocol.html
[RelativeHybridTopologyProtocolSettings]: https://docs.openfree.energy/en/stable/reference/api/openmm_rfe.html#openfe.protocols.openmm_rfe.RelativeHybridTopologyProtocolSettings
[OpenMM]: https://openmm.org

In [2]:
from openfe.protocols.openmm_rfe import (
    RelativeHybridTopologyProtocol,
    RelativeHybridTopologyProtocolSettings,
)
from openfe.protocols.openmm_rfe import equil_rfe_settings

## Configure Protocol Settings

### From the Defaults

The user-configurable settings of a `Protocol` are stored in a separate object that inherits from `ProtocolSettings`. The default settings object for a protocol can be retrieved with the `Protocol.default_settings` class method:

In [3]:
settings = RelativeHybridTopologyProtocol.default_settings()

The settings object is a Pydantic data class, and so can be edited and inspected in the usual ways. For example, the default settings can be printed clearly as a dictionary:

In [4]:
settings.dict()

{'forcefield_settings': {'constraints': 'hbonds',
  'rigid_water': True,
  'hydrogen_mass': 3.0,
  'forcefields': ['amber/ff14SB.xml',
   'amber/tip3p_standard.xml',
   'amber/tip3p_HFE_multivalent.xml',
   'amber/phosaa10.xml'],
  'small_molecule_forcefield': 'openff-2.0.0',
  'nonbonded_cutoff': 1.0 <Unit('nanometer')>,
  'nonbonded_method': 'PME'},
 'thermo_settings': {'temperature': 298.15 <Unit('kelvin')>,
  'pressure': 0.9869232667160129 <Unit('standard_atmosphere')>,
  'ph': None,
  'redox_potential': None},
 'protocol_repeats': 3,
 'solvation_settings': {'solvent_model': 'tip3p',
  'solvent_padding': 1.2 <Unit('nanometer')>},
 'partial_charge_settings': {'partial_charge_method': 'am1bcc',
  'off_toolkit_backend': 'ambertools',
  'number_of_conformers': None,
  'nagl_model': None},
 'lambda_settings': {'lambda_functions': 'default', 'lambda_windows': 11},
 'alchemical_settings': {'softcore_LJ': 'gapsys',
  'explicit_charge_correction_cutoff': 0.8 <Unit('nanometer')>,
  'endstate

The production simulations could be lengthened from 5 ns to 10 ns:

In [5]:
settings.simulation_settings.production_length = 10.0 * unit.nanosecond

### From Scratch

Alternatively, settings can be specified by hand when creating the settings object:

In [6]:
settings = RelativeHybridTopologyProtocolSettings(
    protocol_repeats=3, # Run n independent repeats of each transformation
    forcefield_settings=equil_rfe_settings.OpenMMSystemGeneratorFFSettings(
        constraints='hbonds',              # 'hbonds': Use constraints for bonds involving hydrogen
        rigid_water=True,                  # True: Use constraints for bonds in water
        hydrogen_mass=3.0,                 # Perform hydrogen mass repartitioning
        forcefields=[                      # OpenMM force fields to use for solvents and proteins
            'amber/ff14SB.xml',
            'amber/tip3p_standard.xml',
            'amber/tip3p_HFE_multivalent.xml',
            'amber/phosaa10.xml'
        ],
        
        # Small molecule force field to use with OpenMM template generator:
        small_molecule_forcefield='openff-2.0.0',
        nonbonded_method='PME',            # Particle Mesh Ewald for long range electrostatics
        nonbonded_cutoff=1.0 * unit.nm,    # Cut off Lennard-Jones interactions beyond 1 nm
    ),
    thermo_settings=equil_rfe_settings.ThermoSettings(
        temperature=298.15 * unit.kelvin,  # Set thermostat temperature
        pressure=1 * unit.bar,             # Set barostat pressure
        ph=None,                           # None: Do not keep pH constant
        redox_potential=None               # None: Do not keep redox potential constant
    ),
    solvation_settings=equil_rfe_settings.OpenMMSolvationSettings(
        solvent_model='tip3p',             # Solvent model to generate starting coords
        solvent_padding=1.2 * unit.nm,     # Total distance between periodic image starting coords
    ),
    lambda_settings=equil_rfe_settings.LambdaSettings(
        lambda_functions='default',        # Interpolation functions for force field parameters
        lambda_windows=11,                 # Split the transformation over n lambda windows
    ),
    partial_charge_settings=equil_rfe_settings.OpenFFPartialChargeSettings(
        partial_charge_method='am1bcc',    # Method for partial charge generation
        off_toolkit_backend='ambertools',  # Use ambertools backend for partial charge generation
        number_of_conformers=None,         # The existing conformer of the input ligand will be used
        nagl_model=None,                   # not using NAGL, but am1bcc for partial charge assignment
    ),
    alchemical_settings=equil_rfe_settings.AlchemicalSettings(
        unsampled_endstates=False,         # False: Use only the explicit lambda windows λ∈[0,-1]
        use_dispersion_correction=False,   # False: Use LJ dispersion correction only at endpoints
        softcore_LJ='gapsys',               # Use LJ potential from Gapsys et al. (JCTC 2012)
        softcore_alpha=0.85,               # Set soft-core Lennard-Jones potential parameter α
        turn_off_core_unique_exceptions=False, # False: Keep all exceptions (1,4 or otherwise) at all λ
        flatten_torsions=False,            # False: Keep all torsions at all lambda windows
    ),
    engine_settings=equil_rfe_settings.OpenMMEngineSettings(
        compute_platform=None,              # Let OpenMM choose the best platform for your hardware
    ),
    integrator_settings=equil_rfe_settings.IntegratorSettings(
        timestep=4 * unit.femtosecond,         # Integration timestep
        langevin_collision_rate=1.0 / unit.picosecond, # Langevin integration collision rate γ
        reassign_velocities=False,             # False: Velocities are not lost through MCMC moves
        n_restart_attempts=20,                 # Restart simulations the first n times they blow up
        constraint_tolerance=1e-06,            # Tolerance for holonomic constraints
        barostat_frequency=25 * unit.timestep, # Attempt MC volume scaling every n integration steps
        remove_com=False,                      # False: Do not remove center of mass motion
    ),
    simulation_settings=equil_rfe_settings.MultiStateSimulationSettings(
        minimization_steps=5000,                    # Minimize potential energy for n steps
        equilibration_length=1.0 * unit.nanosecond, # Simulation time to equilibrate for
        production_length=5.0 * unit.nanosecond,    # Simulation time to collect data for
        # H-REX Sampling settings
        sampler_method='repex',            # Sample lambda with Hamiltonian Replica Exchange
        n_replicas=11,                     # Number of HREX replicas with sampler_method='repex'
        
        # # SAMS sampling settings
        # sampler_method='sams',             # Sample lambda with Self Adjusted Mixture Sampling
        # flatness_criteria='logZ-flatness', # Criteria for asymptotically optimal sampling
        # gamma0=1.0,                        # Initial adaptation rate w/ sampler_method='SAMS'
        
        time_per_iteration=1 * unit.picosecond, # Attempt an MCMC move every picosecond
        real_time_analysis_interval=250 * unit.picosecond, # Compute & write out free energies every 250 ps.
        real_time_analysis_minimum_time=500 * unit.picosecond, # Compute & write out free energies only after 500 ps.
        
        # Don't stop sampling early, no matter how low the estimated error gets:
        early_termination_target_error=0.0 * unit.kilocalorie_per_mole
        # # Stop sampling when estimated error is small enough:
        # early_termination_target_error=0.12 * unit.kilocalorie_per_mole,
    ),
    output_settings=equil_rfe_settings.MultiStateOutputSettings(
        output_filename='simulation.nc',            # Filename to save trajectory
        output_structure='hybrid_system.pdb',       # Filename to save starting coordinates
        checkpoint_storage_filename='checkpoint.chk', # Filename for simulation checkpoints
        forcefield_cache='db.json',                 # Cache for small molecule residue templates
        output_indices='not water',                 # Do not save water positions
        checkpoint_interval=1 * unit.picosecond,    # Save a checkpoint every picosecond
    )
)

In [7]:
# Double check that the above settings match the defaults - delete this cell if you configure things yourself!
assert settings == RelativeHybridTopologyProtocol.default_settings()

## Construct the `Protocol`

However you produce the `ProtocolSettings` object, the final `Protocol` can be constructed from the settings object:

In [8]:
protocol = RelativeHybridTopologyProtocol(settings)

Unlike `ProtocolSettings`, a `Protocol` instance is immutable. The only way to safely change the settings of a `Protocol` is to recreate it from the modified `ProtocolSettings` object.